## **Enunciado:**
El objetivo de este proyecto es intentar recuperar una serie de contraseñas que han sido almacenadas de manera segura utilizando 4 protocolos diferentes listados a continuación:
1. Protocolo de almacenamiento de contraseñas versión 1.
   - Se almacena `(username, H(password))`.
2. Protocolo de almacenamiento de contraseñas versión 2.
   - Se almacena `(username, salt, H(password || salt))`.
3. Protocolo de almacenamiento de contraseñas versión 3.
   - Se almacena `(username, salt, H(password || pepper || salt))`.
4. Protocolo de almacenamiento de contraseñas utilizando una PBKDF.
   - Se almacena `(username, salt, PBKDF(password, salt, 64, 2 ** 20, H))`.
     - `64` es el número de bytes que se obtentrán de la función `PBKDF`.
     - `2 ** 20` es la dificultad `d` de la función `PBKDF`.

En todos los escenarios anteriores la función hash `H` utilizada fue `SHA3_512` de `Crypto.Hash`. Para el último escenario la función PBKDF utilizada fue `PBKDF2` de `Crypto.Protocol.KDF`.

En cada escenario deberán obtener la contraseña para cada uno de los integrantes del grupo de trabajo utilizando como identificador el usuario Uninorte. Para ello, deberán utilizar el **archivo de contraseñas comunes** ([rockyou.txt](https://www.kaggle.com/datasets/wjburns/common-password-list-rockyoutxt)) provisto y la base de datos (archivo de contraseñas) correspondiente.

Se recomienda paralelizar el proceso de búsqueda de contraseñas para reducir el tiempo de ejecución.

In [1]:
from Crypto.Hash import SHA3_512
import pandas as pd

In [2]:
def cargar_contraseñas(archivo):
    """Carga las contraseñas desde un archivo de texto y retorna una lista, manejando errores de decodificación."""
    contraseñas = []
    with open(archivo, 'r', encoding='ISO-8859-1') as f:  # abre el archivo en modo binario
        for line in f:
            contraseñas.append(line.strip())
    return contraseñas

In [3]:
def cargar_base_de_datos(filepath):
    """Carga la base de datos de usuarios y contraseñas hash desde un archivo CSV."""
    return pd.read_csv(filepath)

In [4]:
# Carga las contraseñas comunes desde el archivo rockyou.txt
contraseñas = cargar_contraseñas('rockyou.txt')

In [5]:
print(f"Total de contraseñas cargadas: {len(contraseñas)}")

Total de contraseñas cargadas: 14344391


## **Caso 1 (Protocolo V.1):**

In [6]:
def generar_rainbow_table(contraseñas):
    """Genera una rainbow hash table usando SHA3_512."""
    rainbow_table = {}
    for contraseña in contraseñas:
        hash_obj = SHA3_512.new()
        hash_obj.update(contraseña.encode('ISO-8859-1'))
        hash_hex = hash_obj.hexdigest()
        rainbow_table[hash_hex] = contraseña
    return rainbow_table

In [7]:
def buscar_hash_usuario(data, username):
    """Busca el hash de la contraseña para un username específico."""
    return data[data['username'] == username]['password'].values[0]

In [8]:
def buscar_contraseña(hash_hex, rainbow_table):
    """Busca una contraseña en la rainbow hash table basada en un hash SHA3_512."""
    return rainbow_table.get(hash_hex, "Contraseña no encontrada")

In [9]:
def mostrar_contraseña(data_usuarios, usuario, rainbow_table):
    # Busca el hash de la contraseña para el usuario
    hash_usuario = buscar_hash_usuario(data_usuarios, usuario)

    # Busca la contraseña usando el hash encontrado
    contraseña_recuperada = buscar_contraseña(hash_usuario, rainbow_table)
    print(f"Contraseña recuperada para {usuario}:", contraseña_recuperada)

In [10]:
# Genera la rainbow hash table
rainbow_table = generar_rainbow_table(contraseñas)

In [12]:
# Carga el archivo CSV con la base de datos de usernames y contraseñas hash
filename = 'password_databases/password_database_v1.csv'
data_usuarios = cargar_base_de_datos(filename)

In [13]:
mostrar_contraseña(data_usuarios, 'jachaverra', rainbow_table)

Contraseña recuperada para jachaverra: drumsdw


In [14]:
mostrar_contraseña(data_usuarios, 'apmichelle', rainbow_table)

Contraseña recuperada para apmichelle: 4brylend


In [15]:
mostrar_contraseña(data_usuarios, 'alvarobarrios', rainbow_table)

Contraseña recuperada para alvarobarrios: 4querer0AMAR


In [16]:
mostrar_contraseña(data_usuarios, 'santiagocadavid', rainbow_table)

Contraseña recuperada para santiagocadavid: mmh2203


## **Caso 2 (Protocolo V.2):**

In [18]:
filename = 'password_databases/password_database_v2.csv'
db_v2 = cargar_base_de_datos(filename)

In [19]:

# Función para generar el hash SHA3_512 de una contraseña con su salt
def hash_contraseña(contraseña, salt):
    hash_obj = SHA3_512.new()
    hash_obj.update((contraseña).encode('ISO-8859-1') + salt)
    return hash_obj.hexdigest()

# Función para buscar la contraseña en la base de datos usando el hash y el salt
def buscar_contraseña_en_db(db, username, contraseñas_comunes):
    user_data = db[db['username'] == username].iloc[0]
    salt = user_data['salt']
    salt_h = bytes.fromhex(salt)
    hash_pw = user_data['password']
    for contraseña in contraseñas_comunes:
        if hash_contraseña(contraseña, salt_h) == hash_pw:
            return contraseña
    return "Contraseña no encontrada."

In [20]:
def mostrar_contraseña(db, usuario, contraseñas):
    # Asegúrate de que se cargó correctamente la base de datos y la lista de contraseñas
    if db is not None and contraseñas:
        # Buscar la contraseña para el usuario 'apmichelle'
        contraseña_recuperada = buscar_contraseña_en_db(db, usuario, contraseñas)
        print(f"Contraseña recuperada para {usuario}: {contraseña_recuperada}")
    else:
        print("No se pudo cargar la base de datos o la lista de contraseñas comunes.")

In [21]:
mostrar_contraseña(db_v2, 'jachaverra', contraseñas)

Contraseña recuperada para jachaverra: b-rob04


In [24]:
mostrar_contraseña(db_v2, 'apmichelle', contraseñas)

Contraseña recuperada para apmichelle: rodelmar


In [25]:
mostrar_contraseña(db_v2, 'alvarobarrios', contraseñas)

Contraseña recuperada para alvarobarrios: gayeto


In [26]:
mostrar_contraseña(db_v2, 'santiagocadavid', contraseñas)

Contraseña recuperada para santiagocadavid: blondejess


## **Caso 3 (Protocolo V.3):**

In [27]:
#filename = 'password_database_v3.csv'

## **Caso 4 (Protocolo V.PBKDF):**

In [28]:
#filename = 'password_database_vPBKDF.csv'